In [2]:
import numpy as np
import pandas as pd
from scipy.stats import moment
from scipy.optimize import lsq_linear
import matplotlib.pyplot as plt

order_of_polynom: int = 6

def find_matrix_dispersion(df_calib: pd.DataFrame) -> float:
    df: pd.DataFrame = df_calib
    zenith_df: pd.Series = df.loc[:, ['zenith angle']].drop_duplicates(ignore_index=True)
    radius_col: pd.DataFrame = df.loc[:, ['radius sens']]
    amount_unique_cells: int = len(zenith_df)
    number_azim_steps: int = len(radius_col) // amount_unique_cells
    dispersion_list: list = []
    dispersion: float = 0.0
    for i in range(amount_unique_cells):
        dispersion: float = round(float(moment(radius_col[i*number_azim_steps:(i+1)*number_azim_steps], 2)), 3)
        dispersion_list.append(dispersion)
    dispersion_df: pd.DataFrame = pd.DataFrame({'radius dispersion' : dispersion_list})
    dispersion_df = zenith_df.join(dispersion_df)
    result_df: pd.DataFrame = df.merge(dispersion_df, left_on='zenith angle', right_on='zenith angle')
    return result_df

def find_zenith_coax(df_calib: pd.DataFrame) -> float:
    '''Находим соосное расположение ротатора по вертикальной оси'''
    df: pd.DataFrame = df_calib
    dispersion: np.ndarray = df['radius dispersion'].to_numpy()
    min_dev: float = np.min(dispersion)
    zenith_coax_angle: float = df.loc[df['radius dispersion'] == min_dev, 'zenith angle'].iloc[0]
    return zenith_coax_angle

def find_coax_sens(df_calib: pd.DataFrame) -> float:
    df: pd.DataFrame = df_calib[3:]
    zenith_coax_angle: float = find_zenith_coax(df)
    zen_coax: float = np.average(df.loc[df['zenith angle'] == zenith_coax_angle, 'zen sens'].to_numpy())
    azim_coax: float = np.average(df.loc[df['zenith angle'] == zenith_coax_angle, 'azim sens'].to_numpy())
    return zen_coax, azim_coax

def define_aprox_coefs(df_calib: pd.DataFrame) -> list:
    '''Находим коэффициенты апроксимирующего полинома шестой степени из калибровочных данных'''
    df: pd.DataFrame = df_calib
    radius: list = df['radius sens'].to_numpy()
    thetta: list = df['zenith angle'].to_numpy()
    A: list = np.vstack([radius**i for i in range(order_of_polynom + 1)]).T
    coefficients: list = lsq_linear(A, thetta).x
    return coefficients

def define_aprox_polynom(df_calib: pd.DataFrame) -> np.poly1d:
    '''Находим полином шестой степени по коэффициентам'''
    df: pd.DataFrame = df_calib
    calib_coefs: list = define_aprox_coefs(df)
    calib_polynomial: np.poly1d = np.poly1d(np.flip(calib_coefs))
    return calib_polynomial

def find_angle_error(df_calib: pd.DataFrame) -> np.ndarray:
    '''Находим отклонение теоретических данных от экспериментальных'''
    df: pd.DataFrame = df_calib
    calib_polynom: np.poly1d = define_aprox_polynom(df)
    theor_data: np.ndarray = df['radius sens'].to_numpy()
    exp_dat: np.ndarray = df['zenith angle'].to_numpy()
    deviation_buff: np.ndarray = calib_polynom(theor_data) - exp_dat
    return deviation_buff

In [3]:
new_df: pd.DataFrame = pd.read_csv(r'D:\Python_projects\ACSpy\utils\IntensityTables\06-27-2023_17-59-53_Intensity.csv', delimiter=' ')
new_df = new_df.set_axis(['zen sens', 'azim sens', 'x sens', 'y sens', 'zenith angle', 'azimuth angle', 'radius sens'], axis=1)
result: pd.DataFrame = find_matrix_dispersion(new_df)
find_coax_sens(result), find_angle_error(new_df)


((238.0, 242.25),
 array([ 0.1572197 , -0.0497552 , -0.0497186 , -0.21932763, -0.13835093,
         0.16092893,  0.05873938,  0.13362023, -0.14339216,  0.12792592,
         0.0478687 ,  0.08910642, -0.17335143,  0.08556622,  0.00569057,
        -0.19135182, -0.10750515,  0.14046385,  0.05654028, -0.16806857,
        -0.09093935,  0.13759493,  0.06055687,  0.12143849, -0.08282684,
         0.11893161,  0.05613917,  0.07500068, -0.09760969, -0.15851086,
         0.01824169,  0.14473134, -0.00773644, -0.03759308, -0.04868695,
         0.0819952 , -0.00954371, -0.04242974, -0.04719419]))

In [33]:
# moment([3.39E+02, 3.40E+02, 3.41E+02, 3.41E+02], 2), moment([0.55, 1.34, 0.90, 0.90], 2)
# np.sqrt((2.39E+02 - np.mean([2.38E+02, 2.38E+02, 2.37E+02, 2.39E+02]))**2 + (2.42E+02 - np.mean([2.42E+02, 2.42E+02, 2.43E+02, 2.43E+02]))**2)
df_zero = pd.read_csv(r'D:\Python_projects\ACSpy\utils\IntensityTables\06-27-2023_17-59-53_Intensity.csv', delimiter=' ')
df_wo_zero = pd.read_csv(r'D:\Python_projects\ACSpy\utils\IntensityTables\06-27-2023_17-45-35_Intensity.csv', delimiter=' ')
df_zero = df_zero.set_axis(['zen sens', 'azim sens', 'x sens', 'y sens', 'zenith angle', 'azimuth angle', 'radius sens'], axis=1)
df_wo_zero = df_wo_zero.set_axis(['zen sens', 'azim sens', 'x sens', 'y sens', 'zenith angle', 'azimuth angle', 'radius sens'], axis=1)
zero_d = find_matrix_dispersion(df_zero[3:])
wo_zero_d = find_matrix_dispersion(df_wo_zero[4:])
zero_d, wo_zero_d

(    zen sens  azim sens  x sens  y sens  zenith angle  azimuth angle  \
 0      237.0      257.0     NaN     NaN           6.0       0.000141   
 1      253.0      243.0     NaN     NaN           6.0      90.000000   
 2      239.0      227.0     NaN     NaN           6.0     180.000000   
 3      223.0      242.0     NaN     NaN           6.0     270.000000   
 4      237.0      274.0     NaN     NaN          12.0      -0.000176   
 5      269.0      243.0     NaN     NaN          12.0      90.000000   
 6      239.0      211.0     NaN     NaN          12.0     180.000000   
 7      207.0      241.0     NaN     NaN          12.0     270.000000   
 8      237.0      291.0     NaN     NaN          18.0      -0.000053   
 9      286.0      243.0     NaN     NaN          18.0      90.000000   
 10     239.0      194.0     NaN     NaN          18.0     180.000000   
 11     190.0      241.0     NaN     NaN          18.0     270.000000   
 12     237.0      308.0     NaN     NaN          2

In [16]:
import numpy as np
x = [2.38E+02, 2.38E+02, 2.37E+02, 2.39E+02, 2.39E+02, 2.37E+02, 2.53E+02, 2.39E+02, 2.23E+02]
y = [2.42E+02, 2.42E+02, 2.43E+02, 2.43E+02, 2.42E+02, 2.57E+02, 2.43E+02, 2.27E+02, 2.42E+02]
np.linalg.norm([x, y], axis=0)

array([339.42303988, 339.42303988, 339.43777044, 340.83720454,
       340.12497703, 349.59691074, 350.79623715, 329.62099448,
       329.07901787])

In [52]:
import numpy as np
import pandas as pd
from scipy.stats import moment
from scipy.optimize import lsq_linear
import matplotlib.pyplot as plt
from optical_stand.calibrator.data_processing.file_processing import read_calib_from_csv

dispersion_moment: int = 2
number_of_digits: int = 3

def find_matrix_dispersion(df_calib: pd.DataFrame | str) -> pd.DataFrame:
    if isinstance(df_calib, pd.DataFrame):
        df: pd.DataFrame = df_calib
    elif isinstance(df_calib, str):
        df: pd.DataFrame = read_calib_from_csv(df_calib)
    else:
        raise Exception("Wrong DataFrame type")
    zenith_df: pd.Series = df.loc[:, ['zenith angle']].drop_duplicates(ignore_index=True)
    radius_col: pd.DataFrame = df.loc[:, ['radius sens']]
    amount_unique_cells: int = len(zenith_df)
    number_azim_steps: int = len(radius_col) // amount_unique_cells
    dispersion_list: list = []
    dispersion: float = 0.0
    for i in range(amount_unique_cells):
        dispersion: float = round(float(moment(radius_col[i*number_azim_steps:(i+1)*number_azim_steps], dispersion_moment)), number_of_digits)
        dispersion_list.append(dispersion)
    dispersion_df: pd.DataFrame = pd.DataFrame({'radius dispersion' : dispersion_list})
    dispersion_df = zenith_df.join(dispersion_df)
    result_df: pd.DataFrame = df.merge(dispersion_df, left_on='zenith angle', right_on='zenith angle')
    return result_df

def find_zenith_coax(df_calib: pd.DataFrame | str) -> float:
    '''Находим соосное расположение ротатора по вертикальной оси'''
    df: pd.DataFrame = df_calib
    df: pd.DataFrame = find_matrix_dispersion(df)
    dispersion: np.ndarray = df['radius dispersion'].to_numpy()
    min_dev: float = np.min(dispersion)
    zenith_coax_angle: float = df.loc[df['radius dispersion'] == min_dev, 'zenith angle'].iloc[0]
    return zenith_coax_angle

def find_coax_x_y(df_calib: pd.DataFrame | str) -> float:
    '''Находим зенитный и азимутальный центр матрицы'''
    df: pd.DataFrame = df_calib
    zenith_coax_angle: float = find_zenith_coax(df)
    x_coax: float = np.average(df.loc[df['zenith angle'] == zenith_coax_angle, 'x sens'].to_numpy())
    y_coax: float = np.average(df.loc[df['zenith angle'] == zenith_coax_angle, 'y sens'].to_numpy())
    return x_coax, y_coax

def get_center_coords(df_calib: pd.DataFrame | str) -> dict:
    df: pd.DataFrame = df_calib
    x, y = find_coax_x_y(df)
    center_coords_dict: dict = {
        'zenith coax' : find_zenith_coax(df),
        'x coax' : x,
        'y coax' : y
    }
    return center_coords_dict
df = pd.read_csv(r'D:\Python_projects\ACSpy\utils\Intensity_tables\06-27-2023_17-45-35_Intensity.csv', delimiter=' ')
df = df.set_axis(['x sens', 'y sens', 'zen sens', 'azim sens', 'zenith angle', 'azimuth angle', 'radius sens'], axis=1)

image_df = df.head(4)
zenith_col_num: int = image_df.columns.get_loc('zenith angle')
x_sens_col_num: int = image_df.columns.get_loc('x sens')
y_sens_col_num: int = image_df.columns.get_loc('y sens')
image_buffer = image_df.to_numpy()
coax_coord = get_center_coords(df)
for i, sublist in enumerate(image_buffer):
    for j, elem in enumerate(sublist):
        if j == zenith_col_num:
            image_buffer[i][j] = image_buffer[i][j] - coax_coord['zenith coax']
        elif j == x_sens_col_num:
            image_buffer[i][j] = image_buffer[i][j] - coax_coord['x coax']
        elif j == y_sens_col_num:
            image_buffer[i][j] = image_buffer[i][j] - coax_coord['y coax']
image_buffer

# result = [sublist - coax_coord_reshaped for sublist in image_buffer_reshaped]
# result

array([[-2.50e-01, -5.00e-01,       nan,       nan, -1.00e+00,  3.34e-04,
         3.39e+02],
       [-1.25e+00,  5.00e-01,       nan,       nan, -1.00e+00,  9.00e+01,
         3.40e+02],
       [ 7.50e-01,  5.00e-01,       nan,       nan, -1.00e+00,  1.80e+02,
         3.41e+02],
       [ 7.50e-01, -5.00e-01,       nan,       nan, -1.00e+00,  2.70e+02,
         3.40e+02]])

In [29]:

a = np.array([2.38e+02, 2.42e+02, 'nan', 'nan', 0.00e+00, 3.34e-04, 3.39e+02])
a = np.where(a =='nan', 0, a.astype(float))  # Заменяем 'nan' на 0 и преобразуем в числовой тип данных

b = np.array([1, 2])

result = a - np.pad(b, (0, len(a) - len(b)), mode='constant')
print(result)

[2.37e+02 2.40e+02      nan      nan 0.00e+00 3.34e-04 3.39e+02]


<>:2: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:2: SyntaxWarning: "is" with a literal. Did you mean "=="?
C:\Users\Dennis\AppData\Local\Temp\ipykernel_2220\4213960839.py:2: SyntaxWarning: "is" with a literal. Did you mean "=="?
  a = np.where(a is 'nan', 0, a.astype(float))  # Заменяем 'nan' на 0 и преобразуем в числовой тип данных


In [13]:
a = np.array([1, 2, 3, 'nan'])
b = np.array([1, 2])

# Изменяем размерность массива b, чтобы он соответствовал массиву a
b_reshaped = np.pad(b, (0, len(a) - len(b)), mode='constant')

# result = a - b_reshaped
print(b_reshaped)

[ 0.  0.  2. -2.]
